In [3]:
# Resavamo Travelling Salesman Problem

In [9]:
!pip install networkx

                                              0.0/1.7 MB ? eta -:--:--
                                              0.0/1.7 MB ? eta -:--:--
                                              0.0/1.7 MB 1.3 MB/s eta 0:00:02
     -                                        0.1/1.7 MB 656.4 kB/s eta 0:00:03
     --                                       0.1/1.7 MB 901.1 kB/s eta 0:00:02
     -----                                    0.2/1.7 MB 1.4 MB/s eta 0:00:02
     ----------                               0.4/1.7 MB 2.0 MB/s eta 0:00:01
     ------------------                       0.8/1.7 MB 2.9 MB/s eta 0:00:01
     --------------------------------         1.4/1.7 MB 4.4 MB/s eta 0:00:01
     ---------------------------------------  1.7/1.7 MB 5.1 MB/s eta 0:00:01
     ---------------------------------------- 1.7/1.7 MB 4.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import math
import networkx as nx
import random
from copy import deepcopy
from matplotlib import pyplot as plt

In [4]:
def euclid_distance(u_x: float,
                    u_y: float,
                    v_x: float,
                    v_y: float) -> float:
    return math.sqrt((u_x-v_x)**2 + (u_y*v_y)**2)

In [11]:
# svaki cvor je grad, svaka ivica je razdaljina (weight)
# pheromones - ?

def read_instance(file_path: str) -> nx.Graph:
    with open(file_path, 'r') as f:
        g = nx.Graph()
        for line in f:
            if line[0].isdigit():
                node_id, x, y = [float(a) for a in line.split()]
                g.add_node(int(node_id) - 1, x=x, y=y)
                
            for u in range(len(g.nodes)):
                for v in range(u+1, len(g.nodes)):
                    weight = euclid_distance(g.nodes[u]['x'],
                                             g.nodes[u]['y'],
                                             g.nodes[v]['x'],
                                             g.nodes[v]['y'])
                    g.add_edge(u, v,
                              weight=weight,
                              pheromones=0.1)
                    
        return g

In [16]:
def traverse(
    g: nx.Graph,
    start_node: int,
    alpha: float,
    beta: float
):
    cycle = [start_node]
    cycle_weight = 0

    visited = {start_node}
    u = start_node
    
    while len(visited) != len(g.nodes):
        # uzimamo komsije koje nisu posecene
        neighbors = [v for v in g[u] if v not in visited]
        # vrednost svakog komsije (pheromones^alpha / razdaljina^beta)
        values = [g[u][v]['pheromones']**alpha / g[u][v]['weight']**beta for v in neighbors]
        
        # biramo 1 komsiju na osnovu value-a (to je verovatnoca)
        chosen_neighbor = random.choices(neighbors, weights=values, k=1)[0]
        
        # dodajemo u ciklus, posecene
        cycle.append(chosen_neighbor)
        visited.add(chosen_neighbor)
        # azuriram tezinu kruga i prelazim na sledeci cvor
        cycle_weight += g[u][chosen_neighbor]['weight']
        u = chosen_neighbor

    # razdaljina izmedju prvog i poslednjeg
    cycle_weight += g[start_node][cycle[-1]]['weight']
    return cycle, cycle_weight

In [20]:
def aco(
    g: nx.Graph,
    num_ants: int,
    rho: float,
    num_iters: int,
    theta: float,
    alpha: float,
    beta: float
):
    
    best_cycle_lens = []
    final_cycle = None
    
    # napravimo cikluse (to su nam mravi)
    for i in range(num_iters):
        cycles = [traverse(g, start_node=0, alpha=alpha, beta=beta) for _ in range(num_ants)]
    
        # rho - evaporation rate, koliko feromona ispari u svakoj iteraciji
        for u in range(len(g.nodes)):
            for v in range(u+1, len(g.nodes)):
                g[u][v]['pheromones'] *= rho
        
        
        for cycle, cycle_weight in cycles:
            # theta - pheromone update
            delta = theta/cycle_weight
            # updateujemo feromone, zip(svi sem poslednjeg, svi sem prvog)
            for u, v in zip(cycle[:-1], cycle[1:]):
                g[u][v]['pheromones'] += delta
            
            # prvi i poslednji update feromona
            g[cycle[0]][cycle[-1]]['pheromones'] += delta
        
        # biramo najbolji ciklus od ponudjenih: cycles -> (0 cycle, 1 cycle_weight)
        best_cycle = min(cycles, key=lambda c: c[1])
        best_cycle_lens.append(best_cycle[1])
        
        if final_cycle is None or best_cycle[1] < final_cycle[1]:
            final_cycle = deepcopy(best_cycle)
    
    plt.plot(range(num_iters), best_cycle_lens)
    print(final_cycle[0])
    print(final_cycle[1])

In [21]:
g = read_instance(file_path='dj38.tsp')
aco(g=g, num_ants=10, rho=0.9, num_iters=100, alpha=0.9, beta=1.5, theta=100.0)

FileNotFoundError: [Errno 2] No such file or directory: 'dj38.tsp'